In [ ]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "2024-09-25"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

instrument = "LATISS"
observation_reason = "ALL"
observation_type = "ALL"
science_program = "ALL"

In [ ]:
import os
import datetime as dt
import pandas as pd

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        "TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP."
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = "Imported lsst.ts.logging_and_reporting from local packages."
import lsst.ts.logging_and_reporting.source_adapters as sad
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md

# Set default env to "usdf" and try before PUSH to repo.

# The default provided here is for local testing.
# Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", ut.Server.summit
)  # TODO try with "usdf" before push (else "summit")

# Normalize Parameters (both explicit Times Squares params and implicit ones)
observation_reason = None if observation_reason == "ALL" else observation_reason
observation_type = None if observation_type == "ALL" else observation_type
science_program = None if science_program == "ALL" else science_program
params = dict(
    day_obs=day_obs,
    number_of_days=number_of_days,
    instrument=instrument,
    observation_reason=observation_reason,
    observation_type=observation_type,
    science_program=science_program,
)
display(params)

In [ ]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

In [ ]:
src_exp = sad.ExposurelogAdapter(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    limit=5000,
)

In [ ]:
df = src_exp.exposure_detail(
    instrument=instrument,
    science_program=science_program,
    observation_reason=observation_reason,
    observation_type=observation_type,
)
md(f"The number of exposures in this filtered result is {len(df.index)}")
df.to_html()

-----------------

# Developer Only Section

In [ ]:
print(src_exp)
print(f"{instrument=}, {science_program=}, {observation_reason=}, {observation_type=}")

In [ ]:
print({k: len(v) for k, v in src_exp.exposures.items()})